In [ ]:
"""Config"""

import configparser

config = configparser.ConfigParser()
config.read('config.ini')

base_url = config['CONFIG']['URL']
email = config['CONFIG']['Email']
password = config['CONFIG']['Password']
project_id = config['CONFIG']['ProjectId']

In [ ]:
"""Start Selenium WebDriver. Open ChatDoc Master and Login."""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

driver.get(base_url + '/SignIn')

# Locate login fields and login
email_input  = driver.find_element(By.XPATH, "//input[@type='email']")
password_input = driver.find_element(By.XPATH, "//input[@type='password']")
signin_button = driver.find_element(By.XPATH, '//button[contains(text(), "Sign In")]')

email_input.send_keys(email)
password_input.send_keys(password)
signin_button.click()

print('Logged in')

In [ ]:
"""Go to Chatbot Page and wait for WS Connection"""

import time

# Go To Project Page
driver.get(base_url + "/ConversationalAgent/" + project_id)
time.sleep(4)
# Check we have permission to view chatbot.
no_permission = driver.find_elements(By.XPATH, '//*[contains(text(), "Cannot access the chat due to permission denied.")]')
if no_permission:
    raise(BaseException(f"ERROR: No permission to access conversation page {project_id}."))
# Check we can connect to the chatbot.
for i in range(20):
    print(f"Connecting attempt {i + 1}")
    reconnecting = driver.find_elements(By.CLASS_NAME, 'bot-reconnect')
    if not reconnecting:
        break
    time.sleep(1)
    if i == 19:
        raise(BaseException(f"ERROR: Could not connect to conversation page for {project_id}."))
    
print('Connected')

In [ ]:
"""Define Input File Parsers.

Parsers should return the following dict format:
{
    'action' (Literal['send', 'clear']): action to perform
    'message' (str): message to send if action is send
}
"""

import json

def conversation_parser(input_path: str):
    """Parse CDM exported conversations json."""
    with open(input_path) as datafile:
        # May be issues if json file is too large?
        data = json.load(datafile)
        _user_id, _project_id = data['user_id'], data['project_id']
        for message in data['messages']:
            if message['role'] == 'user':
                yield {'action': 'send', 'message': message['content']}
            elif message['role'] == 'system' and message['content'] == 'reset_context':
                yield {'action': 'clear'}

def txt_parser(input_path: str):
    """Parse txt file with one input per line."""
    with open(input_path) as data:
        for line in data:
            line = line.strip()
            if line == '{clear_context}':
                yield {'action': 'clear'}
            else:
                yield {'action': 'send', 'message': line}

# FIXME: Replace with other parsers to support other input formats
input_parser = txt_parser
# FIXME: Change input file, make sure to use correct input parser
input_file = 'data/example_txt.txt'

In [ ]:
"""Question Asking Loop"""

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import json
import re

input_generator = (input_parser(input_file))

# Input textbox and clear_context buttons
input_box = driver.find_element(By.XPATH, '//textarea[@placeholder="Ask something."]')
clear_context = driver.find_element(By.CLASS_NAME, 'icon-clear-line')

# Begin question asking
question_count = 0
for input_dict in input_generator:
    # Wait and confirm message generation
    _thumb_or_reccomened = WebDriverWait(driver, 60).until(EC.visibility_of_any_elements_located((
        By.XPATH,
        # Last bot message's thumbs up message
        '(//div[contains(@class, "bot-message")])[last()]//i[contains(@class, "bi-hand-thumbs-up")]'
        # OR Reccomended questions
        + '| //i[contains(@class, "bi-chevron-right")]'
    )))
    time.sleep(1)

    # Perform Action
    if input_dict['action'] == 'send':
        question = input_dict['message']
        # Remove newlines from question to prevent early sending
        question = re.sub('\n', '', question)
        input_box.send_keys(question)
        print(f'Question {question_count}: {input_box.get_attribute("value")}')
        input_box.send_keys(Keys.ENTER)
        question_count += 1
    elif input_dict['action'] == 'clear':
        clear_context.click()
        time.sleep(2)
        print('Context Cleared')
    else:
        raise KeyError('Invalid action from input generator')
    
print("Question Asking Complete")